# 均值回归策略

## 📋 概述

均值回归策略（Mean Reversion Strategy）是基于"价格会回归均值"这一假设的交易策略。该策略认为，当价格偏离均值（或正常水平）过远时，会向均值回归。本章节将详细介绍均值回归策略的原理、实现方法和实战应用，包括基于布林带、RSI等指标的均值回归策略。

**学习方式**：本文件是Jupyter Notebook格式，你可以边看边运行代码，通过实际数据回测和可视化，深入理解均值回归策略。


In [ ]:
# 安装必要的库（如果还没有安装）
# !pip install pandas numpy matplotlib mplfinance akshare

# 导入库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplfinance as mpf
import akshare as ak
from datetime import datetime, timedelta

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

print("环境准备完成！")


## 📖 核心内容

### 1. 均值回归策略原理

#### 1.1 基本理念

**核心假设**：
- 价格会围绕均值波动
- 当价格偏离均值过远时，会向均值回归
- 适合震荡市场，不适合趋势市场

**策略特点**：
- 在价格低于均值时买入（超卖）
- 在价格高于均值时卖出（超买）
- 适合波动性较大的市场
- 需要设置止损和止盈

#### 1.2 常用技术指标

均值回归策略常用的技术指标包括：
- **布林带（Bollinger Bands）**：判断价格偏离程度
- **RSI**：判断超买超卖
- **KDJ**：判断超买超卖
- **Z-Score**：统计意义上的偏离度

让我们先获取数据，然后实现均值回归策略：


In [ ]:
# 获取股票数据
try:
    stock_data = ak.stock_zh_a_hist(symbol="000001", period="daily", 
                                     start_date="20230101", end_date="20241231", 
                                     adjust="qfq")
    
    stock_data.columns = ['日期', '开盘', '收盘', '最高', '最低', '成交量', '成交额', '振幅', '涨跌幅', '涨跌额', '换手率']
    stock_data['日期'] = pd.to_datetime(stock_data['日期'])
    stock_data = stock_data.set_index('日期')
    stock_data = stock_data[['开盘', '最高', '最低', '收盘', '成交量']]
    stock_data.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
    stock_data = stock_data.sort_index()
    
    print("数据获取成功！")
    
except Exception as e:
    print(f"数据获取失败：{e}")
    print("使用模拟数据演示...")
    dates = pd.date_range(start='2023-01-01', end='2024-12-31', freq='D')
    dates = dates[dates.weekday < 5]
    
    np.random.seed(42)
    # 生成震荡市场的模拟数据
    base_price = 10
    prices = []
    for i in range(len(dates)):
        # 围绕均值震荡
        price = base_price + np.sin(i * np.pi / 20) * 2 + np.random.randn() * 0.3
        prices.append(price)
    
    stock_data = pd.DataFrame({
        'Open': prices + np.random.randn(len(dates)) * 0.05,
        'High': prices + np.abs(np.random.randn(len(dates)) * 0.1),
        'Low': prices - np.abs(np.random.randn(len(dates)) * 0.1),
        'Close': prices + np.random.randn(len(dates)) * 0.05,
        'Volume': np.random.randint(1000000, 10000000, len(dates))
    }, index=dates)
    
    stock_data['High'] = stock_data[['Open', 'Close', 'High']].max(axis=1)
    stock_data['Low'] = stock_data[['Open', 'Close', 'Low']].min(axis=1)
    
    print("使用模拟数据演示")


In [ ]:
# 布林带均值回归策略
def calculate_bollinger_bands(data, period=20, num_std=2):
    """计算布林带"""
    middle = data['Close'].rolling(window=period).mean()
    std = data['Close'].rolling(window=period).std()
    upper = middle + (std * num_std)
    lower = middle - (std * num_std)
    return upper, middle, lower

def bollinger_mean_reversion_strategy(data, period=20, num_std=2):
    """布林带均值回归策略"""
    df = data.copy()
    
    # 计算布林带
    df['BB_Upper'], df['BB_Middle'], df['BB_Lower'] = calculate_bollinger_bands(df, period, num_std)
    
    # 生成交易信号
    df['Signal'] = 0
    df['Position'] = 0
    
    # 价格触及下轨：买入
    df.loc[df['Close'] <= df['BB_Lower'], 'Signal'] = 1
    # 价格触及上轨：卖出
    df.loc[df['Close'] >= df['BB_Upper'], 'Signal'] = -1
    # 价格回归中轨：平仓
    df.loc[(df['Close'] >= df['BB_Middle']) & (df['Position'] == 1), 'Signal'] = 0
    df.loc[(df['Close'] <= df['BB_Middle']) & (df['Position'] == -1), 'Signal'] = 0
    
    # 计算持仓状态
    position = 0
    for i in range(len(df)):
        if df['Signal'].iloc[i] == 1:
            position = 1  # 买入
        elif df['Signal'].iloc[i] == -1:
            position = -1  # 卖出
        elif df['Signal'].iloc[i] == 0 and position != 0:
            position = 0  # 平仓
        df.iloc[i, df.columns.get_loc('Position')] = position
    
    return df

# 应用策略
bb_strategy = bollinger_mean_reversion_strategy(stock_data)

# 可视化策略
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 12), sharex=True,
                                gridspec_kw={'height_ratios': [3, 1]})

# 绘制K线图和布林带
recent_data = bb_strategy.tail(200)
mpf.plot(recent_data[['Open', 'High', 'Low', 'Close', 'Volume']], 
         type='candle', style='charles', ax=ax1, volume=False)

# 绘制布林带
ax1.plot(recent_data.index, recent_data['BB_Upper'], 
         label='上轨', linewidth=1.5, color='red', linestyle='--')
ax1.plot(recent_data.index, recent_data['BB_Middle'], 
         label='中轨', linewidth=1.5, color='blue')
ax1.plot(recent_data.index, recent_data['BB_Lower'], 
         label='下轨', linewidth=1.5, color='green', linestyle='--')

# 填充布林带区域
ax1.fill_between(recent_data.index, recent_data['BB_Upper'], recent_data['BB_Lower'], 
                alpha=0.1, color='gray')

# 标注买卖信号
buy_signals = recent_data[recent_data['Signal'] == 1]
sell_signals = recent_data[recent_data['Signal'] == -1]

ax1.scatter(buy_signals.index, buy_signals['Low'] * 0.98, 
           marker='^', color='green', s=200, label='买入信号', zorder=5)
ax1.scatter(sell_signals.index, sell_signals['High'] * 1.02, 
           marker='v', color='red', s=200, label='卖出信号', zorder=5)

ax1.set_title('布林带均值回归策略', fontsize=16, fontweight='bold')
ax1.set_ylabel('价格')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 绘制持仓状态
ax2.fill_between(recent_data.index, 0, recent_data['Position'], 
                alpha=0.3, color='green', label='持仓状态')
ax2.set_ylabel('持仓')
ax2.set_xlabel('日期')
ax2.set_ylim(-1.1, 1.1)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("布林带均值回归策略说明：")
print(f"买入信号数量：{len(buy_signals)}")
print(f"卖出信号数量：{len(sell_signals)}")


### 3. RSI均值回归策略

RSI（相对强弱指标）也是常用的均值回归指标。

#### 3.1 策略原理

**RSI指标**：
- RSI > 70：超买，考虑卖出
- RSI < 30：超卖，考虑买入
- RSI在30-70之间：正常波动

让我们实现RSI均值回归策略：


In [ ]:
# RSI均值回归策略
def calculate_rsi(data, period=14):
    """计算RSI指标"""
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def rsi_mean_reversion_strategy(data, period=14, oversold=30, overbought=70):
    """RSI均值回归策略"""
    df = data.copy()
    
    # 计算RSI
    df['RSI'] = calculate_rsi(df, period)
    
    # 生成交易信号
    df['Signal'] = 0
    df['Position'] = 0
    
    # RSI < 30：买入
    df.loc[df['RSI'] < oversold, 'Signal'] = 1
    # RSI > 70：卖出
    df.loc[df['RSI'] > overbought, 'Signal'] = -1
    # RSI回归正常：平仓
    df.loc[(df['RSI'] >= 50) & (df['Position'] == 1), 'Signal'] = 0
    df.loc[(df['RSI'] <= 50) & (df['Position'] == -1), 'Signal'] = 0
    
    # 计算持仓状态
    position = 0
    for i in range(len(df)):
        if df['Signal'].iloc[i] == 1:
            position = 1
        elif df['Signal'].iloc[i] == -1:
            position = -1
        elif df['Signal'].iloc[i] == 0 and position != 0:
            position = 0
        df.iloc[i, df.columns.get_loc('Position')] = position
    
    return df

# 应用策略
rsi_strategy = rsi_mean_reversion_strategy(stock_data)

# 可视化策略
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 12), sharex=True,
                                gridspec_kw={'height_ratios': [3, 1]})

# 绘制K线图
recent_data = rsi_strategy.tail(200)
mpf.plot(recent_data[['Open', 'High', 'Low', 'Close', 'Volume']], 
         type='candle', style='charles', ax=ax1, volume=False)

# 标注买卖信号
buy_signals = recent_data[recent_data['Signal'] == 1]
sell_signals = recent_data[recent_data['Signal'] == -1]

ax1.scatter(buy_signals.index, buy_signals['Low'] * 0.98, 
           marker='^', color='green', s=200, label='买入信号', zorder=5)
ax1.scatter(sell_signals.index, sell_signals['High'] * 1.02, 
           marker='v', color='red', s=200, label='卖出信号', zorder=5)

ax1.set_title('RSI均值回归策略', fontsize=16, fontweight='bold')
ax1.set_ylabel('价格')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 绘制RSI指标
ax2.plot(recent_data.index, recent_data['RSI'], label='RSI', linewidth=1.5, color='blue')
ax2.axhline(y=70, color='red', linestyle='--', linewidth=1, label='超买线(70)')
ax2.axhline(y=30, color='green', linestyle='--', linewidth=1, label='超卖线(30)')
ax2.fill_between(recent_data.index, 70, 100, alpha=0.1, color='red', label='超买区域')
ax2.fill_between(recent_data.index, 0, 30, alpha=0.1, color='green', label='超卖区域')
ax2.set_title('RSI指标', fontsize=14, fontweight='bold')
ax2.set_ylabel('RSI')
ax2.set_xlabel('日期')
ax2.set_ylim(0, 100)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("RSI均值回归策略说明：")
print(f"买入信号数量：{len(buy_signals)}")
print(f"卖出信号数量：{len(sell_signals)}")


### 4. 均值回归策略的优缺点

#### 4.1 优点

1. **适合震荡市场**：在横盘震荡市场中表现好
2. **风险可控**：有明确的买入和卖出点
3. **交易频率适中**：不会过于频繁交易
4. **逻辑简单**：容易理解和实现

#### 4.2 缺点

1. **趋势市场表现差**：在强趋势市场中容易亏损
2. **假突破风险**：可能遇到价格继续偏离的情况
3. **需要止损**：必须设置止损点
4. **参数敏感**：需要根据市场调整参数

#### 4.3 改进方法

1. **结合趋势过滤**：只在震荡市场使用均值回归
2. **多指标确认**：结合多个指标提高准确性
3. **动态止损**：根据波动性调整止损点
4. **仓位管理**：根据偏离程度调整仓位

## 💡 关键要点总结

1. **均值回归的核心**：价格会回归均值
2. **常用指标**：布林带、RSI、KDJ
3. **适用市场**：震荡市场，不适合趋势市场
4. **风险控制**：必须设置止损和止盈

## 🛠️ 工具与软件

### Python量化工具

本Notebook使用的工具：
- **pandas**：数据处理
- **numpy**：数值计算
- **matplotlib/mplfinance**：可视化
- **AKShare**：数据获取

### 注意事项

- ✅ **代码可运行**：所有代码都可以直接运行
- ✅ **数据获取**：如果AKShare获取失败，代码会自动使用模拟数据
- ✅ **网络要求**：AKShare通常不需要梯子
- ⚠️ **策略优化**：需要根据实际市场情况调整参数
- ⚠️ **风险控制**：实盘交易必须设置止损

## 🔗 相关知识点

- [震荡指标](../02_技术指标/震荡指标.ipynb)
- [布林带](../02_技术指标/趋势指标.ipynb)
- [RSI指标](../02_技术指标/震荡指标.ipynb)
- [趋势跟踪策略](./趋势跟踪策略.ipynb)
- [突破策略](./突破策略.ipynb)

## 📚 拓展阅读

- 《量化交易：如何建立自己的算法交易》- 均值回归策略详解
- 量化交易平台文档 - 学习量化回测方法
